# Priority queue with heap

Develop a class `MyFirstPriorityQueue` that implements a heap-based priority queue. 

## Notes

Conceivably, we could assemble a collection of `Node` objects beginning with one on top, then two children, each with two children etc. creating an actual binary tree:

![](images/MaxHeapTree.jpg)

If every `Node` has two children except for the nodes in the bottom layer, we can predict the total number of nodes $N$ in a structure with $L$ layers:

$$ 2^{L-1} \leq N \leq 2^L-1 $$

The problem here is that while the tree looks neat when we draw it, its data are scattered all over the memory. There is a more efficient way to get things done in a much simpler manner, using an array (or list) with size $2^L-1$. All we need is to be able to tell where, in the array, are the two children of a node and, conversely, where is the parent of each node. This is easy to accomplish after we rearrange the nodes of the tree on a line, where the the top node is first, nodes of the next layer from left to right are next, and so on. The linear arrangement is shown below.

![](./images/linear_arrangement.jpg)

If we use an array to hold these nodes, the parent-children relation can be expressed as simple algebraic functions of the array indices. For example, after we observe a few relations between parent and children indices, we can reach the conclusion shown in the table below.

|   $p$    |  $c_L$   |  $c_R$   |
| :------: | :------: | :------: |
|    0     |    1     |    2     |
|    1     |    3     |    4     |
|    2     |    5     |    7     |
| $\vdots$ | $\vdots$ | $\vdots$ |
|   $i$    |  $2i+1$  |  $2i+2$  |

So we can write the functions that generate the index of left and write children as

$$
\begin{align*}
c_L(p) & = 2p+1 \\
c_R(p) & = 2p+2 \\ & = c_L(p) + 1
\end{align*}
$$

Or, in plain Python

```python
def left_child(parent: int) -> int:
    return 2*parent + 1

def right_child(parent: int) -> int:
    return 2*parent + 2
```

Conversely, we can find the parent of every node using the inverse relation:

$$
p(c) = \left\lfloor \dfrac{c+1}{2}\right\rfloor
$$

and in plain Python:

```python
def parent(child: int) -> int:
    return (child + 1) // 2
```

With these relations in place, we can write functions `violation_occurs` and `restore` from earlier as array operations. In the code below, we assume that the methods have access to a the `underlying` list with the linear arrangement of the tree. The methods below  assume a **maximum heap** arrangement.
```python

def violation_occurs(parent: int) -> bool:
    """Check whether the heap property is violated at the given parent index."""
    return (
        underlying[left_child(parent)] > underlying[parent]
        and underlying[right_child(parent)] > underlying[parent]
    )

def restore(parent: int) -> None:
    """Restore the heap property at the given parent index."""
    # Find the largest of the children
    left_idx = left_child(parent)
    if underlying[left_idx] is not None:
        # Proceed only if there is at least one child. If there is only one child,
        # it must be the left child. Assign largest to left child initially.
        largest: int = left_idx
        right_idx: int = right_child(parent)
        if underlying[right_idx] is not None and underlying[right_idx] > underlying[left_idx]:
            # If there is a right child, and it is larger than the left child,
            # assign largest to right child instead.
            largest = right_idx
        # Swap
        underlying[parent], underlying[largest] = underlying[largest], underlying[parent]
        return largest # where the parent node ended at

```

### Fast removal

The main feature of this data structure is that it keeps the most important elemenet in its first position and returns it on demand. As shown in the simple implementation above, `remove_most_important` grabs whatever is at position `[0]` of the `underlying` list. In that simple implementation we look for the next important item in the remaining list, to place it to the front, so that we'll be ready for the next request.

While it takes only one step to return the most important element among $n$ elements (because its location is known), it takes up to $n-1$ steps to find the next important element and bring it to the front.

There is a way to do this faster, in no more than $\log_2n$ steps. This is illustrated below.

![](./images/maxheap_removal.jpg)

In the structure above, the most important element is on the top. When we request it, the structure removes and returns it, and replaces it with its last element as shown in part (a). Then, beginning from the top node that just got updated, we check if meets the other feature of this data structure, that a parent node $p$ is at least as important as both its children $c_L$ and $c_R$. In this example, important is measured by greater value, i.e., $p \geq \max\left(c_L,c_R\right)$.

Everytime $p < \min\left(c_L, c_R\right)$, we swap the parent with the largest of the two children and repeat the process until the property is fully restored. This is shown in parts (b), (c), and (d) above. There will be up to $L-1$ swaps, where $L$ is the number of layers. In a data structure with $n$ elements there are $\left\lceil \log_2n\right\rceil$ layers.

Given the list-based implementation, the comparisons between parent and children nodes can done with list indices. This is shown in the code below, which makes use of methods sketched out earlier.

```python
def restore_from_top():
    parent = 0
    while violation_occurs(parent):
        parent = restore(parent)
```


### Fast addition

Addition of new data can also be modeled on the tree structure. New data are added to the last available position of the tree. In the example below, we assume that the last node on the bottom layer was vacant, and we inserted a new value. 

![](./images/maxheap_addition.jpg)

We then proceed to compare the newly placed node with its parent. If the heap property is violated (i.e., `violation_occurs` returns a `True`) we swap and repeat until the property is restored.


## Assignment Part I: implementation

Your class should perform in $\mathcal O(\log_2n)$ time. You may store just plain integer values in it. It is up to you if you design is as a minimum heap -- i.e., the smaller value is always first -- or as maximum heap. Your design choice should be mentioned explicitly in the class's docstring. 

### Specifications 

* The class should implement the following methods:
  * `__init__`
  * `__bool__(self) -> bool` to return False when the queue is empty.
  * `__str__(self) -> str` to return a user-friendly string representation of the class using f-strings.
  * `__len__(self) -> int` to return a non-negative integer with the number of items stored in the object.
  * `is_empty(self) -> bool` as a wrapper for `__bool__`.
  * `size(self) -> int` as a wrapper for `__len__`.
  * `add(self, value:int) -> None` to add a value to the priority queue.
  * `extract(self) -> int` to remove and return the most important value from the queue. Based on your earlier design choice, the most important item is the one with the smallest value (minimum heap) or the largest value (maximum heap).
  * `peek(self) -> int` to show (without removing) the most important element in the structure.
  * `peek_next(self) -> int` to show (without removing) the second most important element in the structure.
  * You may add more methods as you see fit.
* Invariants:
  * After `add` or `extract`, the element at `self._underlying[0]` should always be the most important element in the structure. Based on your earlier design choice, the most important item is the one with the smallest value (minimum heap) or the largest value (maximum heap).
  * For each position $i$ in the underlying list $U$ with $n$ elements, the heap property must be maintained. 
    * For minimum heaps: $U_i \leq \min\left( U_{2i+1}, U_{2i+2} \right),\quad 0\leq i\leq (n-2)/2$
    * For maximum heaps: $U_i \geq \max\left( U_{2i+1}, U_{2i+2} \right),\quad 0\leq i\leq (n-2)/2$
  * Every time more than half the positions of the `_underlying` list have been emptied, the list should be resized accordingly.


### Requirements

* Treat the methods shown in the notes above as *pseudocode* and tweak as needed.
* All class attributes must be shown private.
* Methods that are shown public should document why they are public.
* Methods that return a value must have one and only one `return` statement.
* Methods that do not return a value should not have any `return` statements at all.
* Use type hints.
* The statements `break` and `continue` cannot be used in your code.
* Every method should have a docstring.
* Methods should be further documented with comments.
* Inline comments should be avoided.
* You may not use any list methods (such as `pop`, `extend`, etc), except `append` and `len`. 
* Your code lines should not exceed a length of 80 characters.
* There should be no magic values. The only literals allowed are the numbers `-1`, `0`, `1` (and their `float` variants), `None`, the empty strings `''` and `""`, and the boolean literals. Everything else must be delegated to constants and variables. The only exception to this rule is numbers that implement mathematic formulae. For example $E=mc^2$ can be written as `E:float = m*c**2` though it may be *smarter* to write `E = m*c*c`.


In [ ]:
class MyFirstPriorityQueue:
    """A heap-based priority queue implementation."""

    def __init__(self):
        """Initialize an empty priority queue."""
        # The underlying data structure is a list that will be maintained as a heap.
        self._underlying = []

## Assignment Part II: you *versus* the machine!

*After* you finish your class, ask your favorite AI agent to produce a similar class -- let's call it `AIPriorityQ`. Compare your code and the code produced by the AI and write a brief review of what the machine did better than you and what you did better than the machine. If you do not have a favorite AI tool, you may want to consider downloading and installing the [Ollama](https://ollama.com/) app to run large language models locally.

## What to submit

* File `MyFirstPriorityQueue.py` (your class).
* File `AIPriorityQ.py` (the AI's class).
* File `me_versus_the_machine.txt` with the brief review of of what the machine did better than you and what you did better than the machine.


